In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

import cmath
import pandas as pd 
import numpy as np

#Data Loading
df=pd.read_csv('gdrive/My Drive/DataMiningProject/movies5000db.csv')

print(df)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
         budget       genres  ... vote_count  Verdict
0     237000000       Action  ...      11800      HIT
1     300000000    Adventure  ...       4500     FLOP
2     245000000       Action  ...       4466     FLOP
3     250000000       Action  ...       9106      HIT
4     260000000       Action  ...       2124     FLOP
...         ...          ...  ...        ...      ...
4798     220000       Action  ...        238     FLOP
4799       9000       Comedy  .

In [2]:
#Data Cleaning
df1 = df.iloc[:,[0,8,12,18,19,20]]
df1

,budget,popularity,revenue,vote_average,vote_count,Verdict
0,237000000,150.437577,2787965087,7.2,11800,HIT
1,300000000,139.082615,961000000,6.9,4500,FLOP
2,245000000,107.376788,880674609,6.3,4466,FLOP
3,250000000,112.312950,1084939099,7.6,9106,HIT
4,260000000,43.926995,284139100,6.1,2124,FLOP
...,...,...,...,...,...,...
4798,220000,14.269792,2040920,6.6,238,FLOP
4799,9000,0.642552,0,5.9,5,FLOP
4800,0,1.444476,0,7.0,6,FLOP
4801,0,0.857008,0,5.7,7,FLOP


In [3]:
#Data Partitioning
train = df1.sample(np.int(np.round(df1.shape[0]*0.8)), replace=True)
test = df1.sample(np.int(np.round(df1.shape[0]*0.2)), replace=True)
print("Training Data:\n",train)
print("\nTest Data:\n",test)

Training Data:
         budget  popularity    revenue  vote_average  vote_count Verdict
1465  34000000  131.815575  348319861           7.0        5371    FLOP
2271         0    0.416179          0           5.0           3    FLOP
3842         0    3.115754          0           5.4          43    FLOP
3739   4000000   35.892034   38251425           7.8        1010     HIT
1746  35000000    6.248273   43650785           5.7          57    FLOP
...        ...         ...        ...           ...         ...     ...
626   66000000   18.599910  189198313           6.6         488    FLOP
3382   7000000    2.932784          0           5.0          15    FLOP
456   68000000  139.575085  211817906           7.4        3946     HIT
3214   9000000   14.171759    2500000           5.7         146    FLOP
3259   8000000    5.069095   12671300           6.5          42    FLOP

[3842 rows x 6 columns]

Test Data:
         budget  popularity    revenue  vote_average  vote_count Verdict
2782  1000

In [4]:
#vector creation
labelClass = train['Verdict']
sep_train = {}
for i in range(train.shape[0]):
		vector = train.iloc[i]
		class_value = vector[-1]
		if (class_value not in sep_train):
			sep_train[class_value] = list()
		sep_train[class_value].append(vector)

#find out mean and variance for each column of separated vector
tempC = sep_train['FLOP']
def meanC(columnD):
  return float(sum(columnD)/(float(len(columnD))))
def stdC(columnD):
  tempMean = meanC(columnD)
  tempSum = 0
  for i in range(0,len(columnD)):
    tempSum = tempSum + (columnD.iloc[i] - tempMean)*(columnD.iloc[i] - tempMean)
  return np.sqrt(tempSum/float(len(columnD)-1))

def parametersCalc(train,labelValue):
  meanRow = list()
  stdRow = list()
  tempData = train[train.Verdict == labelValue].iloc[:,:-1]
  for column in tempData:
    meanRow.append(meanC(tempData[column]))
    stdRow.append(stdC(tempData[column]))
  return meanRow,stdRow
print(parametersCalc(train, 'FLOP'))
print(parametersCalc(train, 'HIT'))

([30612878.535935562, 18.564302518587347, 75602654.25123915, 5.835192069392805, 549.7304832713754], [43125043.68067958, 26.742145958593884, 149593836.80791432, 1.0663024832192542, 959.1930072399587])
([27951590.19543974, 39.58762255700324, 132542833.38762215, 7.46970684039088, 1549.6791530944624], [44878981.81569464, 49.98332453595277, 220059378.44458866, 0.364932108886967, 1903.301967999765])


In [5]:
#Calculate probabilities and predict the results for the test data
def gaussianCalc(x,m,s):
	return (1 / (np.sqrt(2 * np.pi) * s)) * np.exp(-((x-m)**2 / (2 * s**2 )))
 
def predict (testData):
  print(testData)
  meanRowF, stdRowF = parametersCalc(train,'FLOP')
  meanRowH, stdRowH = parametersCalc(train,'HIT')
  countTrue = 0
  countFalse = 0
  confusionMatrix =[[0,0], [0,0]]
  for j in range(0,testData.shape[0]):
    
    pF = 0
    pH = 0

    for i in range(0,testData.shape[1]-1):
      pF = pF + np.log(gaussianCalc(testData.iloc[j,i],meanRowF[i],stdRowF[i]))
      pH = pH + np.log(gaussianCalc(testData.iloc[j,i],meanRowH[i],stdRowH[i]))
    if (pF>pH and testData.iloc[j,-1] == 'FLOP'):
      #actual NO and predicted NO 0,0
      countTrue = countTrue + 1
      confusionMatrix[0][0] = confusionMatrix[0][0] + 1
    elif (pF<pH  and testData.iloc[j,-1] == 'HIT'):
      #actual YES and predicted YES 1,1
      countTrue = countTrue + 1
      confusionMatrix[1][1] = confusionMatrix[1][1] + 1
    elif (pF>pH  and testData.iloc[j,-1] == 'HIT'):
      #actual YES and predicted NO 1,0
      countFalse = countFalse + 1
      confusionMatrix[1][0] = confusionMatrix[1][0] + 1
    else:
      #actual NO and predicted YES 0,1
      countFalse = countFalse + 1
      confusionMatrix[0][1] = confusionMatrix[0][1] + 1
  accuracy = (countTrue/ (countFalse + countTrue))*100
  print ("\nAccuracy:", accuracy)
  print("\nConfusion Matrix:\n           Predicted_NO  Predicted_Yes\nActual_NO     ",confusionMatrix[0][0],"              ", confusionMatrix[0][1], "\nActual_Yes    ",confusionMatrix[1][0],"              ", confusionMatrix[1][1])
  print("\nTrue Positive:", confusionMatrix[1][1])
  print("False Positive:", confusionMatrix[0][1])
  print("True Negative:", confusionMatrix[0][0])
  print("False Negative:", confusionMatrix[1][0])
  precision = confusionMatrix[1][1] / (confusionMatrix[1][1] + confusionMatrix[0][1])  # = tp /tp+fp
  recall = confusionMatrix[1][1] / (confusionMatrix[1][1] + confusionMatrix[1][0]) # = tp /tp+fn
  fMeasure = 2*precision*recall/(precision+recall)
  print("\nPrecision: ",precision)
  print("Recall: ",recall)
  print("\nF-Measure: ",fMeasure)

predict(test)

        budget  popularity    revenue  vote_average  vote_count Verdict
2782  10000000    4.267466     303877           5.8          45    FLOP
2309         0    1.583918          0           5.8          11    FLOP
1040  46000000   28.964162  341631208           5.0         543    FLOP
1849  25000000   24.356483  122489822           6.4         711    FLOP
2023  22000000   13.786075   84772742           4.6         304    FLOP
...        ...         ...        ...           ...         ...     ...
1981  24000000   23.343839   83188165           5.3         581    FLOP
4138    100000    7.447714      95000           5.5          92    FLOP
1720  28000000   45.054936   96188903           7.1        4645     HIT
1199  40000000   41.570047  120175290           7.3        1221     HIT
657   65000000    6.227675  240159255           5.6        1600    FLOP

[961 rows x 6 columns]

Accuracy: 90.53069719042664

Confusion Matrix:
           Predicted_NO  Predicted_Yes
Actual_NO      749       